In [1]:
import numpy as np
import cv2 
import scipy
import matplotlib.pylab as plt
from skimage import io
import os

In [2]:
sift = cv2.SIFT_create()
bf = cv2.BFMatcher()
AruCo_dict = cv2.aruco.Dictionary_get(cv2.aruco.DICT_4X4_100)
AruCo_params = cv2.aruco.DetectorParameters_create()

In [3]:
def write_text(img, pose, dy, text) :
    x0 = pose[0]
    y0 = pose[1]
    for i, line in enumerate(text.split('\n')) :
        y = y0 + i*dy
        cv2.putText(img, line, np.int32([x0, y]), cv2.FONT_HERSHEY_COMPLEX, 0.75, (50,200,255), 2)

In [4]:
def feature_object_detection(template_img, template_gray, query_img, query_gray, min_match_number,im_src,src_points) :
    template_kpts, template_desc = sift.detectAndCompute(template_gray, None)
    query_kpts, query_desc = sift.detectAndCompute(query_gray, None)
    matches = bf.knnMatch(template_desc, query_desc, k=2)
    good_matches = list()
    good_matches_list = list()
    for m, n in matches :
        if m.distance < 0.6*n.distance :
            good_matches.append(m)
            good_matches_list.append([m])
    
    if len(good_matches) > min_match_number :
        src_pts = np.float32([ template_kpts[m.queryIdx].pt for m in good_matches ]).reshape(-1,1,2)
        dst_pts = np.float32([ query_kpts[m.trainIdx].pt for m in good_matches ]).reshape(-1,1,2)

        H, inlier_masks = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 10) # H RANSAC
        # get the bounding box around template image
        h, w = template_img.shape[:2]
        template_box = np.float32([[0, 0], [0, h-1], [w-1, h-1], [w-1, 0]]).reshape(-1,1,2)
        transformed_box = cv2.perspectiveTransform(template_box, H)
        
        template_box2 = np.float32([[0, 0], [0, 354], [354, 354], [354, 0]]).reshape(-1,1,2)
        transformed_box2 = cv2.perspectiveTransform(template_box2, H)


        detected_img = cv2.polylines(query_img, [np.int32(transformed_box)], True, (0,0,255), 3, cv2.LINE_AA)
        

        # Calculate Homography
        h2, status = cv2.findHomography(src_points, transformed_box2)

        # Warp source image to destination based on homography
        warped_image = cv2.warpPerspective(im_src, h2, (frame.shape[1],frame.shape[0]))
            
        # Prepare a mask representing region to copy from the warped image into the original frame.
        mask = np.zeros([frame.shape[0], frame.shape[1]], dtype=np.uint8)
        cv2.fillConvexPoly(mask, np.int32(transformed_box2), (255, 255, 255), cv2.LINE_AA)
        im_out = cv2.add(frame, warped_image, mask=cv2.bitwise_not(mask))
        im_out = cv2.add(im_out, warped_image)

        markerCorners, markerIds, rejectedCandidates = cv2.aruco.detectMarkers(im_out, AruCo_dict, parameters = AruCo_params)
        
        if len(markerCorners) > 0:
            detected_img = cv2.aruco.drawDetectedMarkers(im_out, markerCorners)
            rvecs, tvecs, points = cv2.aruco.estimatePoseSingleMarkers(markerCorners , 0.05, K, dist)
            for (rvec, tvec, id, corner) in zip(rvecs, tvecs, markerIds, markerCorners) :
                detected_img = cv2.aruco.drawAxis(im_out, K, dist, rvec, tvec, 0.05)
                x = tvec[0,0]
                y = tvec[0,1]
                z = tvec[0,2]
                text = "id: {}\n pose:\n {:.3f}\n {:.3f}\n {:.3f}".format(id, x, y, z)
                cX = (corner[0,0][0] + corner[0,2][0]) / 2
                cY = (corner[0,0][1] + corner[0,2][1]) / 2
                x0 = cX
                y0 = cY
                for i, line in enumerate(text.split('\n')) :
                    y = y0 + i*20
                    cv2.putText(detected_img, line, np.int32([x0, y]), cv2.FONT_HERSHEY_COMPLEX, 0.75, (50,200,255), 2)
                
        print(len(markerCorners))
        return detected_img, im_out
    else :
    
        print('Keypoints not enough')
        return query_img, query_img

In [5]:
params_dir = os.getcwd()+'/new-camera_params/'
print(params_dir)

#load camera parameters
K = np.load(params_dir+'K.npy')
dist = np.load(params_dir+'dist.npy')

print("Camera matrix")
print(K)
print("Len distortion")
print(dist)

chessboard_size = (8,6) # Your chessboard size
# iterative termination criteria, maximum iterationm and epsilon
term_criteria = (cv2.TermCriteria_EPS+ cv2.TermCriteria_MAX_ITER, 30, 0.001)
# Defining the world coordinates for 3D points
objp = np.zeros((1, chessboard_size[0] * chessboard_size[1], 3), np.float32)
objp[0,:,:2] = np.mgrid[0:chessboard_size[0], 0:chessboard_size[1]].T.reshape(-1, 2)
# 3D object for visualization
axis = np.float32([[3,0,0], [0,3,0], [0,0,-3]]).reshape(-1,3)
box = np.float32([[0,0,0], [0,4,0], [4,4,0], [4,0,0],
                   [0,0,-4],[0,4,-4],[4,4,-4],[4,0,-4] ])

d:\1-2564\010723305-IMAGE PROCESSING\Final Examination(old)/new-camera_params/
Camera matrix
[[1.29968396e+03 0.00000000e+00 6.32814379e+02]
 [0.00000000e+00 1.29933998e+03 4.74721903e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
Len distortion
[[-0.40795692  0.24741507  0.00200567 -0.00202735 -0.09445671]]


In [6]:
vid = cv2.VideoCapture('final_exam\Dataset-1\left_output-1.avi')

template_img = cv2.imread('final_exam\Templates\Template-3.png') 
template_gray =  cv2.cvtColor(template_img, cv2.COLOR_BGR2GRAY)

im_src = cv2.imread('final_exam\Templates\im_src.jpg')
src_points = np.float32([[0,0], [354,0],[354,354] ,[0,354] ])
while vid.isOpened() :
    ret, frame = vid.read()
    
    if ret :
        frame_gray =  cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        detected,im_out =  feature_object_detection(template_img, template_gray, frame, frame_gray, 10,im_src,src_points)
        cv2.imshow('Video frame', detected)
        cv2.imshow('test',im_out )
        if cv2.waitKey(int(1000/30)) & 0xFF == ord('q') : # this line control the period between image frame
            break
    else :
        break
vid.release()
cv2.destroyAllWindows()

Keypoints not enough
Keypoints not enough
0
0
0
Keypoints not enough
0
0
0
0
0
0
0
0
0
0
0
0
0
Keypoints not enough
Keypoints not enough
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
Keypoints not enough
0
0
0
0
0
Keypoints not enough
0
Keypoints not enough
Keypoints not enough
0
0
0
0
0
0
0
0
0
0
0
0
0
Keypoints not enough
0
0
0
0
0
0
0
0
0
Keypoints not enough
0
0
0
0
0
0
0
0
0
0
0
0
Keypoints not enough
Keypoints not enough
Keypoints not enough
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
Keypoints not enough
Keypoints not enough
0
0
0
0
0
0
0
0
0
0
0
Keypoints not enough
Keypoints not enough
Keypoints not enough
Keypoints not enough
Keypoints not enough
Keypoints not enough
Keypoints not enough
Keypoints not enough
Keypoints not enough
Keypoints not enough
Keypoints not enough
Keypoints not enough
Keypoints not enough
Keypoints not enough
Keypoints not enough
Keypoints not enough
Keypoints not enough
Keypoints not enough
Keypoints not en